In [31]:
import pandas as pd
pd.set_option('display.max_columns', None)

from estudo import Estudo

Estudo(
    ano = 2024,
    id_modulo = None,
    analito = None
).run()

362 Fosfatase Ácida Total


item,part,id_modulo,analito,sistema,A24R1I0,A24R1I1,A24R1I2,A24R1I3,A24R1I4,A24R1I5,A24R1I6,A24R1I7,A24R1I8,A24R1I9,A24R2I0,A24R2I1,A24R2I2,A24R2I3,A24R2I4,A24R2I5,A24R2I6,A24R2I7,A24R2I8,A24R2I9,A24R3I0,A24R3I1,A24R3I2,A24R3I3,A24R3I4,A24R3I5,A24R3I6,A24R3I7,A24R3I8,A24R3I9,A24R4I0,A24R4I1,A24R4I2,A24R4I3,A24R4I4,A24R4I5,A24R4I6,A24R4I7,A24R4I8,A24R4I9
330,25,362,Fosfatase Ácida Total,1,NaN,17.6,4.2,5.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.9,9.7,3.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.4,5.5,3.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.4,2.8,3.7,NaN,NaN,NaN,NaN,NaN,NaN
2001,94,362,Fosfatase Ácida Total,1,NaN,16.9,3.5,4.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.2,9.0,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,5.7,3.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,3.1,3.6,NaN,NaN,NaN,NaN,NaN,NaN
2458,124,362,Fosfatase Ácida Total,1,NaN,16.8,3.4,4.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.6,7.6,2.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.3,4.6,2.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.6,2.9,3.4,NaN,NaN,NaN,NaN,NaN,NaN
2459,124,362,Fosfatase Ácida Total,2,NaN,17.9,4.0,5.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.7,9.4,3.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.7,5.8,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.6,3.1,3.5,NaN,NaN,NaN,NaN,NaN,NaN
32255,3466,362,Fosfatase Ácida Total,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.8,8.9,2.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.4,5.6,3.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.3,2.5,3.2,NaN,NaN,NaN,NaN,NaN,NaN
48209,5596,362,Fosfatase Ácida Total,1,NaN,16.6,3.6,4.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.5,8.8,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.1,4.7,3.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.5,2.8,3.2,NaN,NaN,NaN,NaN,NaN,NaN
52966,6086,362,Fosfatase Ácida Total,1,NaN,18.3,3.7,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.7,7.8,2.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,5.3,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.4,3.0,3.2,NaN,NaN,NaN,NaN,NaN,NaN
65855,7463,362,Fosfatase Ácida Total,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.6,8.5,3.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.3,5.3,3.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.6,2.9,3.4,NaN,NaN,NaN,NaN,NaN,NaN
74657,9379,362,Fosfatase Ácida Total,1,NaN,15.9,3.4,4.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.5,8.4,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,5.1,3.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,3.1,3.2,NaN,NaN,NaN,NaN,NaN,NaN
76169,9466,362,Fosfatase Ácida Total,1,NaN,17.9,4.0,5.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.7,9.4,3.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.7,5.8,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.6,3.1,3.5,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
from utils import Data

data = Data()

# parceiros = data.get_parceiros()
# modulos = data.get_modulos()

In [32]:
df = data.get_data(ano = 2024, id_modulo = [362], analito = ['Fosfatase Ácida Total'])
df.sort_values(['part','envio'])

,part,id_modulo,analito,envio_str,envio,item_ensaio,sistema,valor
10,25,362,Fosfatase Ácida Total,Out/2023,2023-10-01,BQ02,1,4.2
11,25,362,Fosfatase Ácida Total,Out/2023,2023-10-01,BQ01,1,17.6
13,25,362,Fosfatase Ácida Total,Out/2023,2023-10-01,BQ03,1,5.3
85,25,362,Fosfatase Ácida Total,Jan/2024,2024-01-01,BQ01,1,8.9
86,25,362,Fosfatase Ácida Total,Jan/2024,2024-01-01,BQ02,1,9.7
...,...,...,...,...,...,...,...,...
54,16637,362,Fosfatase Ácida Total,Abr/2024,2024-04-01,BQ02,1,5.8
58,16637,362,Fosfatase Ácida Total,Abr/2024,2024-04-01,BQ03,1,3.4
129,16637,362,Fosfatase Ácida Total,Jul/2024,2024-07-01,BQ01,1,8.7
135,16637,362,Fosfatase Ácida Total,Jul/2024,2024-07-01,BQ02,1,2.7


In [33]:
from utils import PreProcessamento

prepross = PreProcessamento(df)
df_tratado = prepross.get_data()
df_tratado

item,part,id_modulo,analito,sistema,A24R1I1,A24R1I2,A24R1I3,A24R2I1,A24R2I2,A24R2I3,A24R3I1,A24R3I2,A24R3I3,A24R4I1,A24R4I2,A24R4I3
0,25,362,Fosfatase Ácida Total,1,17.6,4.2,5.3,8.9,9.7,3.8,2.4,5.5,3.3,9.4,2.8,3.7
1,94,362,Fosfatase Ácida Total,1,16.9,3.5,4.9,8.2,9.0,2.8,3.0,5.7,3.9,9.0,3.1,3.6
2,124,362,Fosfatase Ácida Total,1,16.8,3.4,4.2,6.6,7.6,2.7,2.3,4.6,2.7,8.6,2.9,3.4
3,124,362,Fosfatase Ácida Total,2,17.9,4.0,5.2,7.7,9.4,3.2,2.7,5.8,3.7,8.6,3.1,3.5
5,3466,362,Fosfatase Ácida Total,1,NaN,NaN,NaN,8.8,8.9,2.7,2.4,5.6,3.2,8.3,2.5,3.2
6,5596,362,Fosfatase Ácida Total,1,16.6,3.6,4.3,7.5,8.8,2.8,2.1,4.7,3.3,8.5,2.8,3.2
8,6086,362,Fosfatase Ácida Total,1,18.3,3.7,5.0,6.7,7.8,2.1,2.5,5.3,3.0,8.4,3.0,3.2
9,7463,362,Fosfatase Ácida Total,1,NaN,NaN,NaN,7.6,8.5,3.1,2.3,5.3,3.3,8.6,2.9,3.4
10,9379,362,Fosfatase Ácida Total,1,15.9,3.4,4.3,7.5,8.4,3.0,2.5,5.1,3.3,8.0,3.1,3.2
11,9466,362,Fosfatase Ácida Total,1,17.9,4.0,5.2,7.7,9.4,3.2,2.7,5.8,3.7,8.6,3.1,3.5


In [ ]:
teste = df_tratado.set_index(['part','id_modulo','analito','sistema'])

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler    
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Preenchendo vazios pela média da linha
imputer = SimpleImputer(strategy="mean")
dados: np.ndarray = imputer.fit_transform(teste.T).T

# Aplicando a normalização
norm = StandardScaler()

dados = norm.fit_transform(dados)
pd.DataFrame(dados)

neighbors = NearestNeighbors(n_neighbors=2)
neighbors_fit = neighbors.fit(dados)
distances, _ = neighbors_fit.kneighbors(dados)
distances = np.mean(distances[:,1:], axis=1)
distances = np.sort(distances)
distances = distances[np.where(distances <= 2)]
slopes = np.diff(distances)
# value = slopes[np.where(slopes>=1/100)][0]
slopes[np.where(slopes>=1/100)]

# index = np.argmax(slopes==value)

# eps = distances[index+1]
# eps

array([], dtype=float64)